In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

In [ ]:
# 对每一列进行标准化处理
def cost_normalize(column):
    '''
    成本型指标标准化，越小越好 (越接近1)
    '''
    max_value = column.max()
    min_value = column.min()
    return (max_value - column) / (max_value - min_value)

def benefit_normalize(column):
    '''
    效益型指标标准化，越大越好 (越接近1)
    '''
    max_value = column.max()
    min_value = column.min()
    return (column - min_value) / (max_value - min_value)

def fix_normalize(column, a=0):
    '''
    TODO: seems not correct
    固定型指标标准化，越接近固定值 a 最好 (越接近1)
    '''
    max_value = np.abs(column - a).max()
    min_value = np.abs(column - a).min()
    return (max_value - np.abs(column - a)) / (max_value - min_value)


In [ ]:
# 计算每列的权重
def entropy_weight(column):
    '''
    熵权法计算权重
    '''
    p = column / column.sum()
    e = np.nansum(-p*np.log(p)/np.log(len(column)), axis=0)
    return (1 - e) / np.sum(1 - e)

def coefficient_variation(column):
    '''
    变异系数法计算权重
    '''
    # 计算每列的变异系数
    cv = column.std() / column.mean()
    # 计算每列的权重
    w = cv / np.sum(cv)
    return np.array(w)

In [ ]:
def get_output_dir():
    script_dir = os.getcwd()
    output_dir = os.path.join(script_dir, "output")
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)
    return output_dir


In [ ]:
datafilelist = ['1.经济发展度', '2.社会和谐度', '3.生活方便度', '4.环境舒适度']
yearlist = ['2018', '2019', '2020', '2021']
# datafilelist = ['1.经济发展度']
# yearlist = ['2018']
factorl1list = ['经济发展度', '社会和谐度', '生活方便度', '环境舒适度']
# factors = [['社会消费品零售总额（亿元）', '人均GDP(元)', '一般公共预算收入（亿元）', '一般公共预算支出（亿元）', '第三产业占比'],
        #    ['住户存款余额（亿元）', '城镇非私营单位在岗职工平均工资(元)', '年末户籍人口（万人）', '住宅商品房价格与人均工资比', '城镇登记失业率', '社会抚养比'],
        #    ['年末邮政局（个）', '普通本科专科在校学生数（万人）', '人均移动电话资源占有数(个)', '每千人口拥有执业 (助理) 医师数', '医院数（个 ）', '执业(助理)医师数（人）', '年末户籍人口（千人）', '年末移动电话用户数量（万个）'],
        #    ['道路交通等效声级dB(A)', '环境噪声等效声级dB(A)', '第二产业占比', '城市绿化覆盖率', '第一产业增加值(亿元)', '第三产业增加值（亿元）'],
        #   ]
# 0-效益型，1-成本型，2-中间型
bcfi = [[0, 0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 0],
        ]
benidx = [[0,1,2,3,4], [0,1,2,3], [0,1,2,3,4], [3]]
costidx = [[], [], [], [0,1,2]]
fixidx = [[], [], [], []]

global_weight = [0.33617548, 0.2995685 , 0.25207122, 0.1121848]


In [ ]:

# i=0
# df = pd.ExcelFile(f'./data/世界城市排名数据.xlsx').parse(sheet_name=factorl1list[i], index_col=0)
# df

In [ ]:
# read and build a complete df, need to rename columns, select target year columns, then compute

dfall = pd.DataFrame()
for i in range(0,4):
    nxsh = pd.ExcelFile(f'./data/世界城市排名数据.xlsx').parse(sheet_name=factorl1list[i], index_col=0)
    newcolumns=[]
    for yr in yearlist:
        newcolumns.append(f'{factorl1list[i]}_{yr}')
    print(newcolumns)
    nxsh.columns=newcolumns

    dfall = pd.concat((dfall, nxsh), axis=1)

dfall

In [ ]:
# read and build separate df for each year, also need to rename columns, then compute


In [ ]:
# yr=2018
# yrid=int(yr)-2018
# # df_2018 = df.loc[:, df.columns.contains(['2018'])]
# df_2018 = dfall.filter(regex='2018$', axis="columns")
# # print(df_2018)
# df_2018.columns = factorl1list
# print(df_2018)

In [ ]:

# dfranks = df_2018

def gen_reldevfuz(dfranks):
    # preprocess data
    numerical_features = dfranks.dtypes[dfranks.dtypes != 'object'].index
    # print(numerical_features)

    benefit_index = numerical_features[0:0]
    cost_index = numerical_features[0:4]
    # fix_index = numerical_features[0:0]

    dfranks_benefit = dfranks[benefit_index]
    dfranks_cost = dfranks[cost_index]
    # df_yr_fix = dfranks[fix_index]

    # normalize each column
    dfranks_benefit_new = dfranks_benefit.apply(benefit_normalize)
    dfranks_cost_new = dfranks_cost.apply(cost_normalize)
    # df_yr_fix_new = df_yr_fix.apply(fix_normalize)

    # concat benefit, cost and fix parts
    dfranks_norm = pd.concat((dfranks_benefit_new, dfranks_cost_new), axis=1)
    # now dfranks_norm is the 相对偏差模糊矩阵
    return dfranks_norm



In [ ]:

def gen_rank1yr(reldevfuzarray):
    coevar = coefficient_variation(reldevfuzarray.iloc[:, :])
    # print(coevar)
    # pie_rank(coevar.tolist(), yr)
    # calculate rank
    rank1yr = reldevfuzarray.iloc[:, :].dot(coevar)
    rank1yr.name = yr
    # print(rank1yr)
    # print(type(rank1yr))
    return rank1yr

# sorted_rank1yr = rank1yr.sort_values(ascending=False)
# print(sorted_rank1yr)
# plot_rank(sorted_rank1yr, f'全指标_综合评价_{yr}')


In [ ]:
# print(rank1yr)

dffinalrankbyyr = pd.DataFrame()
for yr in yearlist:
    yrid=int(yr)-2018
    df_yr = dfall.filter(regex=f'{yr}$', axis="columns")
    df_yr.columns = factorl1list
    # print(df_yr)

    reldevfuzarray = gen_reldevfuz(df_yr)

    rank1yr = gen_rank1yr(reldevfuzarray)
    # print(rank1yr)

    dffinalrankbyyr = pd.concat((dffinalrankbyyr, rank1yr.rename(f'{yr}')), axis=1)

print(f'dffinalrankbyyr')
print(dffinalrankbyyr.shape)
print(dffinalrankbyyr)


In [ ]:
sorted_dffinalrankbyyr = dffinalrankbyyr.sort_values(by='2021', ascending=False)

print(f'sorted dffinalrankbyyr')
print(sorted_dffinalrankbyyr)

output_dir = get_output_dir()
with pd.ExcelWriter(os.path.join(output_dir, f'世界城市综合评价.xlsx'), engine='openpyxl', mode='w') as writer:
    sorted_dffinalrankbyyr.to_excel(writer, sheet_name=f'全指标_各年综合评价')

plt.rc('font',family='SimHei')  #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
# plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['figure.figsize'] = (20, 2.5)

plt.plot(sorted_dffinalrankbyyr, 'o-', linewidth=2, alpha=0.5)

# plt.xlabel('xlabel')
plt.ylabel(f'综合评价分数')
plt.legend(labels = yearlist)
output_dir = get_output_dir()
plt.savefig(os.path.join(output_dir, f'世界城市_全指标_各年综合评价.pdf'), bbox_inches='tight')
plt.show()
plt.clf()


In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        pass


In [ ]:
raise StopExecution

In [ ]:
print(f'Should not see me printed!')